## Atividades Desenvolvidas no Notebook - 02_ingestão_camada_bronze

|Sequência|Ação|Detalhamento
|---|---|---|
|SEQ-01 / SEQ-02|Ingestão dos Dados|Identificar no storage "dbfs:/FileStore/tables/landing_zone" os dados a serem inseridos na camada BRONZE.|

In [0]:
diretorio_landing_zone = "dbfs:/FileStore/tables/landing_zone/"
arquivos_diretorio = dbutils.fs.ls(diretorio_landing_zone)

In [0]:

for arquivo in arquivos_diretorio:

    diretorio = arquivo.path
    tabela = arquivo.name.replace('/','').lower()

    parDF1=spark.read.parquet(diretorio)

    colunas = parDF1.columns
    lista1 = ''
    listaInsert = ''
    listaCondicao = ''

    for coluna in colunas:
        if lista1 == '':
            lista1 = coluna
        else:
            lista1 = lista1 + ',' + coluna

    for coluna in colunas:
        if listaInsert == '':
            listaInsert = 'b.'+coluna
        else:
            listaInsert = listaInsert + ',' + 'b.'+ coluna

    for coluna in colunas:
        if listaCondicao == '':
            listaCondicao =  'a.'+coluna + '= b.'+coluna

    if tabela == 'tb_item_venda':
        listaCondicao = listaCondicao + ' AND a.CODIGO_PRODUTO = b.CODIGO_PRODUTO'

    parDF1.createOrReplaceTempView(tabela)

    comando = f"""
                MERGE INTO bronze.{tabela} a
                USING {tabela} b
                ON {listaCondicao}
                WHEN NOT MATCHED
                THEN INSERT ({lista1}) VALUES ({listaInsert})
            """

    print(comando)

    spark.sql(comando)
